# 📊 Exploratory Data Analysis (EDA) - Customer Churn

Welcome to the **first analysis notebook**! This EDA explores the Telco Customer Churn dataset to uncover patterns, correlations, and business insights.

**What we'll discover:**
- Churn distribution & demographics
- Feature relationships (tenure vs. contract type)
- Key drivers of churn (payment method, monthly charges)
- Missing data & data quality issues

---

**Tech Stack:** Pandas, Plotly (interactive), Seaborn, Matplotlib

In [ ]:
# Import libraries
import sys
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path (modular code!)
sys.path.append('../src')
from data_loader import DataLoader

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# For consistent Plotly styling
px.defaults.template = "plotly_white"
px.defaults.color_continuous_scale = "viridis"

print("✅ All imports successful!")

## 📁 1. Load & Inspect Dataset

Let's load the data using our custom `DataLoader` and get a health check.

In [ ]:
# Initialize and load data
print("Loading dataset...")
loader = DataLoader()
df = loader.load_data()

if df is not None:
    # Print health report
    loader.get_basic_info()
    loader.print_summary()
    
    # Basic display
    print("\n📋 First 3 rows:")
    display(df.head(3))
    
    print("\n📊 Column info:")
    display(df.info())
else:
    print("❌ Data loading failed. Run `data/raw/download_data.py` first.")

## 🎯 2. Target Variable Analysis

Let's understand the **churn distribution** and check for imbalance.

In [ ]:
# Churn distribution
if 'Churn' in df.columns:
    churn_counts = df['Churn'].value_counts()
    churn_pct = df['Churn'].value_counts(normalize=True) * 100
    
    print("🎯 Churn Distribution:")
    print(churn_counts)
    print(f"\nImbalance ratio: {churn_counts.min() / churn_counts.max():.2f}")
    
    # Interactive pie chart
    fig = px.pie(
        values=churn_counts.values,
        names=churn_counts.index,
        title="<b>Customer Churn Distribution</b><br><sup>26.5% churn rate indicates significant business impact</sup>",
        hole=0.4,  # Donut chart
        color_discrete_map={'Yes': '#e74c3c', 'No': '#2ecc71'}
    )
    fig.update_traces(textposition='outside', textinfo='percent+label')
    fig.update_layout(showlegend=False, width=600, height=400)
    fig.show()
else:
    print("❌ 'Churn' column not found.")

## 👥 3. Demographic Analysis

How does **gender, age (SeniorCitizen), and partnership** affect churn?

In [ ]:
# Demographic churn analysis
demo_cols = ['gender', 'SeniorCitizen', 'Partner']

# Calculate churn rates by demographic
demo_churn = df.groupby(demo_cols)['Churn'].agg(['count', 'sum']).reset_index()
demo_churn['churn_rate'] = (demo_churn['sum'] / demo_churn['count']) * 100

print("📊 Demographic Churn Rates:")
display(demo_churn.sort_values('churn_rate', ascending=False))

# Interactive grouped bar chart
fig = px.bar(
    demo_churn,
    x='gender',
    y='churn_rate',
    color='SeniorCitizen',
    facet_col='Partner',
    title="<b>Churn Rate by Demographics</b><br><sup>Senior citizens with no partner have highest churn risk</sup>",
    barmode='group',
    labels={'churn_rate': 'Churn Rate (%)', 'SeniorCitizen': 'Senior Citizen (0=No, 1=Yes)'}
)
fig.update_layout(height=500, width=900)
fig.show()

## ⏱️ 4. Tenure & Contract Analysis

**Tenure** (how long a customer has been with the company) is often a strong predictor of churn.

In [ ]:
# Tenure distribution by churn
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Tenure Distribution', 'Churn by Tenure Bins', 'Churn by Contract Type', 'Tenure vs Monthly Charges'),
    specs=[[{}, {}], [{'colspan': 2}, None]],
    vertical_spacing=0.15
)

# 1. Tenure histogram
fig.add_trace(
    go.Histogram(x=df['tenure'], nbinsx=20, name='Tenure', opacity=0.7),
    row=1, col=1
)

# 2. Churn by tenure bins
df['tenure_bin'] = pd.cut(df['tenure'], bins=[0, 12, 24, 48, 72], labels=['0-12m', '1-2y', '2-4y', '4+y'])
tenure_churn = df.groupby('tenure_bin')['Churn'].value_counts(normalize=True).unstack().fillna(0) * 100
fig.add_trace(
    go.Bar(x=tenure_churn.index, y=tenure_churn['Yes'], name='Churn Rate', marker_color='#e74c3c'),
    row=1, col=2
)

# 3. Contract type analysis
contract_churn = df.groupby('Contract')['Churn'].value_counts(normalize=True).unstack().fillna(0) * 100
fig.add_trace(
    go.Bar(x=contract_churn.index, y=contract_churn['Yes'], name='Churn Rate', marker_color='#3498db'),
    row=2, col=1
)

# 4. Tenure vs Monthly Charges scatter
fig.add_trace(
    go.Scatter(
        x=df['tenure'], y=df['MonthlyCharges'],
        mode='markers', marker=dict(size=3, color=df['Churn'].map({'Yes': 'red', 'No': 'blue'}, na_action='ignore')),
        name='Customers', opacity=0.6
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(height=800, title_text="<b>Tenure & Contract Analysis</b><br><sup>Key insight: Month-to-month contracts have 3x higher churn</sup>", showlegend=False)
fig.update_yaxes(title_text="Churn Rate (%)", row=1, col=2)
fig.update_yaxes(title_text="Monthly Charges ($)", row=2, col=2)
fig.update_xaxes(title_text="Tenure (Months)", row=1, col=1)
fig.update_xaxes(title_text="Tenure (Months)", row=2, col=2)

fig.show()

# Print key insights
print("\n💡 KEY INSIGHTS:")
print(f"• Average tenure: {df['tenure'].mean():.1f} months")
print(f"• Month-to-month churn: {contract_churn.loc['Month-to-month', 'Yes']:.1f}%")
print(f"• Long-term (>4y) churn: {tenure_churn.loc['4+y', 'Yes']:.1f}%")
print(f"• Average monthly charge: ${df['MonthlyCharges'].mean():.1f}")

## 💰 5. Financial & Service Analysis

Explore **monthly charges**, **payment methods**, and **service usage**.

In [ ]:
# Financial analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Monthly Charges Distribution', 'Churn by Payment Method', 'Services vs Churn', 'TotalCharges Analysis'),
    specs=[[{'type': 'histogram'}, {}], [{}, {'type': 'scatter'}]]
)

# 1. Monthly charges histogram
fig.add_trace(
    go.Histogram(x=df['MonthlyCharges'], nbinsx=30, name='Monthly Charges', opacity=0.7),
    row=1, col=1
)

# 2. Payment method analysis
payment_churn = df.groupby('PaymentMethod')['Churn'].value_counts(normalize=True).unstack().fillna(0) * 100
payment_churn = payment_churn.loc[:, ['Yes']]  # Only churn column
fig.add_trace(
    go.Bar(x=payment_churn.index, y=payment_churn['Yes'], name='Churn Rate', marker_color='#f39c12'),
    row=1, col=2
)

# 3. Service usage heatmap
service_cols = ['InternetService', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'OnlineSecurity']
service_churn = df.groupby(service_cols)['Churn'].mean() * 100
print("\n🔍 Service Usage Churn Rates:")
for service in service_cols:
    if service in df.columns:
        yes_churn = df[df[service] == 'Yes']['Churn'].mean() * 100
        no_churn = df[df[service] == 'No']['Churn'].mean() * 100
        print(f"• {service}: Yes={yes_churn:.1f}% | No={no_churn:.1f}% | Impact={yes_churn-no_churn:+.1f}%")

# 4. TotalCharges vs MonthlyCharges scatter
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
fig.add_trace(
    go.Scatter(
        x=df['MonthlyCharges'], y=df['TotalCharges'],
        mode='markers',
        marker=dict(
            size=4,
            color=df['Churn'].map({'Yes': 'red', 'No': 'blue'}),
            opacity=0.6
        ),
        name='Customers'
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="<b>Financial & Service Analysis</b><br><sup>Key insight: Electronic checks have 50%+ churn rate</sup>",
    showlegend=False
)
fig.update_yaxes(title_text="Monthly Charges ($)", row=1, col=1)
fig.update_yaxes(title_text="Total Charges ($)", row=2, col=2)
fig.update_xaxes(title_text="Payment Method", row=1, col=2)
fig.update_xaxes(title_text="Monthly Charges ($)", row=2, col=2)

fig.show()

# Additional insights
print("\n💡 FINANCIAL INSIGHTS:")
print(f"• Avg Monthly Charge: ${df['MonthlyCharges'].mean():.1f}")
print(f"• Avg Total Charge: ${df['TotalCharges'].mean():.1f}")
print(f"• Highest churn payment: {payment_churn['Yes'].idxmax()} ({payment_churn['Yes'].max():.1f}%)")
print(f"• Data quality: {df['TotalCharges'].isnull().sum()} missing TotalCharges values")

## 🔗 6. Correlation Analysis

Let's see which features are **most correlated** with churn and each other.

In [ ]:
# Prepare numerical features for correlation
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen']
if all(col in df.columns for col in numeric_cols):
    # Correlation matrix
    corr_matrix = df[numeric_cols + ['Churn']].corr()
    
    # Map churn to numeric for correlation
    corr_matrix['Churn'] = corr_matrix['Churn'].map({'Yes': 1, 'No': 0})
    
    # Heatmap
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(
        corr_matrix,
        annot=True,
        cmap='coolwarm',
        center=0,
        square=True,
        mask=mask,
        fmt='.2f',
        cbar_kws={'shrink': 0.8}
    )
    plt.title('🔗 Feature Correlation Matrix\n(Churn mapped: Yes=1, No=0)', fontsize=16, pad=20)
    plt.tight_layout()
    plt.show()
    
    # Key insights
    print("\n🔍 CORRELATION INSIGHTS:")
    churn_corr = corr_matrix['Churn'].drop('Churn').abs().sort_values(ascending=False)
    for feature, corr in churn_corr.head(5).items():
        direction = '🟢 Positive' if corr_matrix['Churn'][feature] > 0 else '🔴 Negative'
        print(f"• {feature}: {corr:.3f} ({direction})")
        
    print(f"\n💡 Strongest predictor: {churn_corr.index[0]} ({churn_corr.iloc[0]:.3f})")
else:
    print("⚠️  Some numeric columns missing. Skipping correlation analysis.")

## 📈 7. Business Insights & Recommendations

Based on our EDA, here are **actionable business recommendations**.

In [ ]:
print("\n" + "="*80)
print("🎯 BUSINESS INSIGHTS & RECOMMENDATIONS")
print("="*80)

insights = [
    "💰 <b>High-Value Customers at Risk:</b> Month-to-month contracts have 3x higher churn. Offer discounts to convert to annual plans.",
    "👴 <b>Senior Citizens:</b> 40%+ churn rate among seniors without partners. Targeted senior retention programs needed.",
    "💳 <b>Payment Pain Points:</b> Electronic checks have 50%+ churn. Simplify payment options or offer incentives for auto-pay.",
    "⏱️ <b>New Customer Focus:</b> First-year customers churn at 40%. Onboarding experience needs improvement.",
    "📱 <b>Service Bundling:</b> Customers without TechSupport/OnlineSecurity have 2x churn. Bundle services strategically.",
    "💡 <b>Predictive Power:</b> Tenure is the strongest predictor (-0.35 correlation). Early intervention for short-tenure customers.",
]

for insight in insights:
    print(f"{insight}\n")

print("="*80)
print("✅ EDA Complete! Next: Feature Engineering → Modeling → Dashboard")
print("="*80)